In [70]:
import os
import json
import re

def build_training_data(root_folder, output_file):
    """
    Duyệt qua folder 'root_folder' (ví dụ: data) và tạo tập dữ liệu huấn luyện.
    Với mỗi file .txt (đại diện cho một điều luật), chương trình sẽ tạo một bản ghi
    có dạng:
        {
            "instruction": "Nội dung của [Điều ...] trong [Chương ...] của [Tên luật] ([Tên chuyên mục]) là gì?",
            "input": "",
            "output": "[Nội dung của file]",
            "meta": {
                "law_category": "<Tên thư mục cấp cao (ví dụ: bo_luat_dan_su)>",
                "law_file": "<Tên folder chứa file luật (ví dụ: luat_dan_su)>",
                "chapter": "<Tên chương>",
                "clause": "<Tên file (mã hiệu điều)>"
            }
        }
    Kết quả sẽ được lưu vào file output_file dưới định dạng JSON Lines.
    """
    training_examples = []
    
    # Duyệt đệ quy qua các thư mục của folder gốc
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.lower().endswith(".txt"):
                file_path = os.path.join(root, file)
                
                # Đọc nội dung file
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read().strip()
                    
                # Lấy tên file không có phần mở rộng để dùng làm tiêu đề điều
                clause_title = os.path.splitext(file)[0]
                
                # Tách đường dẫn để lấy thông tin meta-data
                # Giả sử cấu trúc: root_folder / law_category / law_file / chapter / <clause file>
                parts = os.path.normpath(file_path).split(os.sep)
                try:
                    # Tìm vị trí của folder gốc (ví dụ: "data")
                    idx = parts.index(os.path.basename(root_folder))
                except ValueError:
                    idx = 0
                    
                # Với cấu trúc trên, giả sử:
                # parts[idx+1] = law_category, parts[idx+2] = law_file, parts[idx+3] = chapter
                if len(parts) >= idx + 4:
                    law_category = parts[idx + 1]
                    law_file = parts[idx + 2]
                    chapter = parts[idx + 3]
                else:
                    law_category = law_file = chapter = ""
                
                # Xây dựng câu hỏi (instruction) cho mô hình
                # Ví dụ: "Nội dung của Điều 43 trong Chương III của luat_lao_dong (bo_luat_lao_dong) là gì?"
                instruction = f"Nội dung của {clause_title} trong {chapter} của {law_file} ({law_category}) là gì?"
                
                # Tạo bản ghi (record)
                record = {
                    "instruction": instruction,
                    "input": "",
                    "output": content,
                    "meta": {
                        "law_category": law_category,
                        "law_file": law_file,
                        "chapter": chapter,
                        "clause": clause_title
                    }
                }
                training_examples.append(record)
    
    # Ghi các bản ghi vào file output (mỗi dòng là 1 JSON object)
    with open(output_file, "w", encoding="utf-8") as out_f:
        for example in training_examples:
            json_line = json.dumps(example, ensure_ascii=False)
            out_f.write(json_line + "\n")
    
    print(f"Đã tạo xong tập training data với {len(training_examples)} bản ghi tại: {output_file}")

if __name__ == "__main__":
    # Đường dẫn folder chứa dữ liệu đã trích xuất (cấu trúc như đã tạo trong folder data)
    data_folder = "data_new"  # hoặc đường dẫn tương ứng nếu khác
    output_training_file = "training_data.jsonl"
    
    build_training_data(data_folder, output_training_file)

Đã tạo xong tập training data với 1115 bản ghi tại: training_data.jsonl
